# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

#### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Realizado por:

* Antonio Beltrán Navarro
* Ramón Jesús Martínez Sánchez

# Pima Diabetes

El objetivo de este _dataset_ es predecir si un paciente tiene diabetes basándose en diversos factores como la cantidad de glucosa, la edad, o el número de embarazos. Todos los pacientes que se incluyen son mujeres de descendencia pima india.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from imblearn import FunctionSampler
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


import utilidad_grupor_practica1 as utils

seed = 27912

Lo primero que haremos es cargar los datos y dividirlos en conjunto de entrenamiento y de test estratificando. En este conjunto de datos no tenemos index por lo que lo indicaremos con `None`.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv" 

index = None
target = "Outcome"

data = utils.load_data(filepath, index, target)

Usaremos un tamaño del conjunto de entrenamiento y de test del 70% y 30% respectivamente.

In [ ]:
(X, y) = utils.divide_dataset(data, target="Outcome")

train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Nos será útil juntar el conjunto de entrenamiento en una sola variable que contenga ambas las variables predictoras y la variable clase.

In [ ]:
train = X_train.copy()
train[target] = y_train

Mostremos que el proceso ha sido satisfactorio mostrando el conjunto de train.

In [ ]:
train.sample(5, random_state=seed)

Lo siguiente que haremos es ver información sobre las variables y sus tipos.

In [ ]:
train.info()

Como podemos observar tenemos una sola variable categórica (la clase) mientras que las predictoras son todas númericas (6 que solo toman valores enteros en el conjunto de datos y 2 con valores reales).

Las variables con las que trabajaremos son:

* Pregnancies: el número de embarazos.
* Glucose: la concentración de glucosa en plasma.
* BloodPressure: la presión arterial diastólica (mmHg).
* SkinThickness: el espesor del pliegue del tríceps (mm).
* Insulin: la insulina en suero.
* BMI: el índice de masa corporal ($\frac{kg}{m^2}$)
* DiabetesPedigreeFunction: función de pedigree de diabetes.
* Age: la edad.
* Outcome: si tiene diabetes o no.

## 1. Análisis exploratorio de datos

### Análisis univariado

Primero veamos cómo se distribuyen las diferentes variables.

In [ ]:
train.describe().T

Aquí ya podemos observar ciertas anomalías, como por ejemplo que existe el valor 0 en variables donde no tiene sentido como `Glucose`, `SkinThickness` o `Insulin`. Además en las dos últimas este valor también marca el primer cuartil lo que indica una gran cantidad de datos erróneos. Por el otro extremo, los valores máximos de algunas variables parecen también ser poco verosímiles y probablemente _outliers_ como es el caso de 17 en `Pregnancies`.

In [ ]:
train.describe(include='category')

En la variable clase no parece haber anomalías en cuanto a su dominio.

In [ ]:
utils.plot_barplot(train)

Sin embargo la cantidad de datos que existen de cada clase no es la misma, por lo que estamos ante un problema desbalanceado.

Mostremos ahora las distribuciones de las distintas variables.

In [ ]:
utils.plot_histogram(train)

Como observamos antes, existen muchos valores perdidos que vienen representados con 0 en variables en las que este valor no está entre los valores razonables para la variable. Es el caso de `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` y `BMI`. Antes de hacer nada más, veamos qué fracción del total de valores están perdidos por variable.

In [ ]:
vs = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
ntrain = train.replace({v: {0: np.NaN} for v in vs})
utils.not_valid_values_plot(ntrain, vs)

Podemos ver que casi un 50% de los valores de la variable `Insulin` y un 30% de la variable `SkinThickness` no son válidos, por lo que hemos decidido eliminarlas al tener demasiado sesgo. Para el resto de variables imputaremos los valores perdidos, por la media en el caso en los tres casos al ser variables que admiten valores reales. Tras esto mostraremos los histogramas otra vez para ver cómo son realmente las distribuciones sin que se vean influenciadas por los valores perdidos.

In [ ]:
train_clean = utils.imputar_valores(train)
utils.plot_histogram(train_clean)

Podemos observar que tenemos lo que parecen distribuciones normales en las variables `Glucose`, `BMI`, `BloodPressure` y `DiabetesPedigreeFunction`, teniendo esta última una asimetría a la derecha destacable. En las variables `Pregnacies` y `Age` podemos observar que los datos siguen una tendencia descendente conforme aumenta el valor de la variable, por lo que se alejan más que el resto de la distribución normal y se acercan a una exponencial.

Además, también podemos observar la existencia de _outliers_ y valores ruidosos en algunas variables como es el caso de `Pregnancies` cuyos valores llegan a alcanzar el valor 17 y que también posee una cantidad considerable entre 6 y 14; lo cual es poco verosímil.

Para ver claramente la posible existencia de _outliers_ en las distribuciones, las visualizaremos mediante un diagrama de cajas.

In [ ]:
f = go.Figure(data=[{'type': 'box', 'y': train_clean[v], 'name': v} for v in set(train_clean.columns) - {"Outcome"}])
f.show()

Tenemos una cantidad considerable de *outliers* sobre todo en las variables `BloodPressure`, `DiabetesPedigreeFunction` y `Age`.

### Análisis multivariado
Ahora realizaremos un análisis multivariado para intentar encontrar correlaciones entre las variables

In [ ]:
fig = utils.plot_pairplot(train_clean, 'Outcome')
fig.update_layout(width=1200, height=1000)

No se puede observar ninguna partición clara de los puntos con ninguna de las variables que nos ayude a decidirnos sobre hacer alguna discretización.

In [ ]:
px.imshow(train_clean.corr())

Podemos ver que `Age` y `Pregnancies` presentan cierta correlación (0.55) aunque no es lo suficientemente significativa para eliminar alguna de las dos. También observamos que las variables predictoras son en general bastante independientes de la variable clase ya que no se puede observar una diferencia clara en la distribución de los puntos etiquetados como 0 y como 1.

Al no poder observar puntos de corte claros, tomaremos la decisión de realizar la discretización en tres intervalos de igual anchura.



In [ ]:
disc = KBinsDiscretizer(n_bins=3, strategy='uniform')

## 2. Preprocesamiento de datos

Primero recopilaremos el preprocesamiento que hemos decidido que queremos hacer en un pipeline.

En primer lugar, aplicaremos una imputación para las variables con valores perdidos que se vieron en el análisis exploratorio y que permanecerán en el conjunto de datos como `BMI, Glucose y BloodPressure`. El método de imputación a seguir será por la media, de modo que estos valores perdidos se verán sustituidos por la media.

Además, eliminaremos las variables que se han comentado previamente en el análisis exploratorio, como son `Insulin y SkinThickness`.

In [ ]:
mea_imp = SimpleImputer(missing_values=0, strategy='mean')
preproc = ColumnTransformer([('', 'drop', ['Insulin', 'SkinThickness']), 
                             ('mea_inp', mea_imp, ["BMI", "Glucose", "BloodPressure"])], remainder='passthrough')

## 3. Aprendizaje y Evaluación *Zero-R*

Empezaremos entrenando un Zero-R. Para cada clasificador mostraremos la matriz de confusión y analizaremos los resultados al final.

In [ ]:
zeror = DummyClassifier(strategy='most_frequent', random_state=seed)

clean_zeror = make_pipeline(preproc, zeror)
utils.evaluate(clean_zeror, X_train, X_test, y_train, y_test)

cfs = [confusion_matrix(y_test, clean_zeror.predict(X_test))]




## 4. Aprendizaje y Evaluación *Árbol de Decisión*

Ahora usaremos un árbol de clasificación sin discretización previa y con los hiperparámetros por defecto:


In [ ]:
ndt = DecisionTreeClassifier(random_state=seed)

cln_ndt = make_pipeline(preproc, ndt)

utils.evaluate(cln_ndt, X_train, X_test, y_train, y_test)

cfs.append(confusion_matrix(y_test, cln_ndt.predict(X_test)))

Por último, discretizaremos antes de entrenar el árbol

In [ ]:
cln_disc_ndt = make_pipeline(preproc, disc, ndt)

utils.evaluate(cln_disc_ndt, X_train, X_test, y_train, y_test)

cfs.append(confusion_matrix(y_test, cln_disc_ndt.predict(X_test)))

In [ ]:
go.Figure([go.Scatter(x=[0, 1], y=[0, 1], line={'dash': 'dash'}, name='Clasificador aleatorio'), go.Scatter(x=[cfs[i][0, 1] / (cfs[i][0, 0] + cfs[i][0, 1]) for i in range(len(cfs))], y=[cfs[i][1, 1] / (cfs[i][1, 1] + cfs[i][1, 0]) for i in range(len(cfs))], mode='markers', hovertext=['ZeroR', 'Árbol sin discretizar', 'Árbol discretizando'], name='Clasificadores propuestos')], layout={'title': 'Espacio ROC', 'xaxis': {'title': '1-specificity'}, 'yaxis': {'title': 'sensitivity'}})

Observando los resultados de los tres clasificadores, podemos decir que los árboles de decisión son superiores al ZeroR en *accuracy*, además de alcanzar un mejor compromiso entre *sensitivity* y *specificity*. Podemos observar estos valores individualmente también y vemos que si nos interesa optimizar el *recall*, que suele ser el caso con ciertos problemas médicos como este, el que mejor valor obtiene en esta métrica es árbol entrenado sin discretizar previamente. Por tanto en este problema podemos observar que la discretización previa no ha sido significativamente útil.

Sin embargo, entre los dos árboles, la discretización previa parece que mejora ligeramente las medidas consideradas cuando consideramos ambas _sensitivity_ y _specificity_ con respecto a no realizarla.

----

# Wisconsin

Comenzamos cargando el conjunto de datos `wisconsin`:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Una vez hemos cargado el conjunto de datos, mostraremos 5 registros aleatorios mediante la función `sample` para comprobar que el proceso ha sido realizado correctamente

In [ ]:
data.sample(5, random_state=seed)

Como podemos observar, tenemos una columna, la última, cuyo nombre es `Unnamed` y todo el contenido de sus filas `NaN`. Esto significa que antes de continuar trabajando con nuestra base de datos, debemos borrarla.

In [ ]:
del data['Unnamed: 32']

Comprobamos que se ha borrado correctamente haciendo uso del método `sample` de nuevo:

In [ ]:
data.sample(5, random_state=seed)

In [ ]:
data.diagnosis.unique()

Es muy útil disponer del conjunto de datos separado dos subconjuntos, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). Se puede utilizar el siguiente fragmento de código para dividirlo: 

A continuación, separaremos en dos subconjuntos nuestro conjunto de datos inicial, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). 

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Comprobaremos que se hayan separado correctamente:

Empezamos mostrando las variables predictoras:

In [ ]:
X.sample(5, random_state=seed)

Y a continuación la variable objetivo:

In [ ]:
y.sample(5, random_state=seed)

Antes de comenzar el análisis exploratorio de los datos, dividiremos nuestro conjunto de datos en otros dos subconjuntos, uno de entrenamiento y otro de prueba, con los siguientes porcentajes:

* Conjunto de entrenamiento: **70%**
* Conjunto de prueba: **30%**

Mediante este proceso, nos aseguraremos de que los resultados posteriores del proceso de validación han sido obtenidos de una manera correcta.

In [ ]:
train_size = 0.7
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Seguidamente, lo que haremos será unir los conjuntos `X_train` e `y_train` para obtener el conjunto de datos de entrenamiento.
Haremos los mismo para `X_test` e `y_test`, juntando así el conjunto de datos de test.

In [ ]:
train = utils.join_dataset(X_train, y_train)
test = utils.join_dataset(X_test, y_test)

Nuevamente, vamos a asegurarnos de que el conjunto de datos se ha dividido correctamente. Comenzamos con las variables del conjunto de datos de entrenamiento, observando que la variable clase también aparece al final del conjunto:

In [ ]:
train.sample(5, random_state=seed)

Comprobamos lo mismo para nuestro nuevo conjunto de prueba:

In [ ]:
test.sample(5, random_state=seed)

Una vez tenemos bien definidos nuestros conjuntos de entrenamiento y prueba, podemos pasar con el análisis exploratorio de datos.

## 1. Análisis exploratorio de datos

El análisis exploratorio de datos es un paso fundamental a la hora de comprender los datos con los que vamos a trabajar.

El objetivo de este análisis es explorar, describir y visualizar la naturaleza de los datos recogidos mediante la aplicación de técnicas simples de resumen de datos y métodos gráficos, para observar las posibles relaciones entre las variables de nuestro conjunto de datos.

Para comenzar, veremos una descripción del conjunto de datos que vamos a emplear.

### Descripción del conjunto de datos
El número de casos y variables (respectivamente) del conjunto de datos se puede obtener consultando el atributo `shape`:
Observamos como de los 569 registros iniciales, tenemos **398** de ellos para el entrenamiento (un 70%), y **171** (un 30%) para el conjunto de prueba, ambos con 31 variables, las 30 predictoras y la variable objetivo.

In [ ]:
data.shape

In [ ]:
train.shape

In [ ]:
test.shape

Para conocer cuál es el tipo de las variables, recurrimos al método `info`:

In [ ]:
train.info(memory_usage=False)

De acuerdo con esto, todas las variables predictoras del conjunto de datos son numéricas (`float64`). Sin embargo, la variable clase (`diagnosis`) es categórica y contiene dos estados, cuyos valores serán `M`y`B` como ya hemos comentado anteriormente:

In [ ]:
y_train.cat.categories

Otra cosa que observamos gracias a aplicar el método `info` sobre nuestro conjunto de datos de entrenamiento es que en realidad solo contamos con **10 variables** reales, pero que se convierten en 30 puesto que están divididas en 3 categorías `mean`, `se` y `worst`. Es importante considerar esto cuando pasemos a visualizar las variables posteriormente, puesto que al haber tal cantidad de variables será mejor una representación dividida por estas 3 categorías.

### Visualización de las variables

A la hora de visualizar las variables, comenzaremos comprobando la distribución de la variable clase de nuestro conjunto de entrenamiento. Como vemos, hay 250 instancias de la clase mayoritaria `B`, y 148 de la clase `M`.

In [ ]:
train.describe(include="category")

Se puede observar mejor de la siguiente manera, comprobando gráficamente qué clase es la que predomina:

In [ ]:
utils.plot_barplot(train)

Lo que podemos observar es que las clases de la variable objetivo del problema no tienen el mismo número de casos, es decir, el problema está desbalanceado (las frecuencias de las combinaciones de estados no aparecen en la misma proporción).

Ahora que sabemos que el problema es **desbalanceado**, es el turno de visualizar las variables predictoras del conjunto de datos.

Comenzaremos mostrando un **histograma** con todas las variables, para mostrar la densidad de ejemplos para los distintos valores de las variables numéricas y analizar las tendencias que estas toman:

In [ ]:
utils.plot_histogram(train)

Mediante el estudio de este histograma en el que se han representado todas las variables, podremos extraer diversas conclusiones que nos serán útiles a la hora de preprocesar los datos de nuestro conjunto:
* Mirando las variables independientemente, encontramos outliers en las variables `radius, area, y perimeter` en sus tres variantes, lo que será algo a tener en cuenta a la hora de preprocesar los datos si queremos eliminarlos.
* Estimación de las distribuciones:
    * Si analizamos las variables `mean`, veremos que `radius, perimeter y area` siguen una distribución normal al igual que la mayoría de variables, excepto las relacionadas con la concavidad, que pueden llegar a tomar una distribución exponencial.
    * Si analizamos las variables `se`, veremos que `radius, perimeter y area` siguen ahora una distribución exponencial, al igual que las relacionadas con la concavidad.
    * Si analizamos las variables `worst`, veremos que `radius, perimeter y area` vuelven a seguir una distribución normal, y las relacionadas con la concavidad pasan a ser distintas, ahora solo `concavity` sigue una tendencia exponencial, mientras que `concave_points` pasa a tomar una clara distribución normal.
* Tras este paso, vemos también que todas las distribuciones tienden a ir hacia la **derecha**.

Ahora realizaremos un análisis multivariado para intentar encontrar correlaciones entre las variables.

Para ello, obtenemos del conjunto de entrenamiento solo las variables de tipo `mean`, puesto que son las más representativas. Con ellas, realizaremos una matriz de dispersión por parejas para comprobar las relaciones entre ellas.

Como podemos ver, la variable clase se diferencia también en nuestra gráfica, siendo el color naranja el diagnóstico `M`, y su homólogo azul el diagnóstico `B`:

In [ ]:
train_mean = train.loc[:,'radius_mean':'fractal_dimension_mean']
train_mean["diagnosis"] = train["diagnosis"]

fig = utils.plot_pairplot(train_mean, 'diagnosis')
fig.update_layout(width=1600, height=1400)

Estudiando la matriz de dispersión de las variables predictoras ordenadas por parejas, podemos sacar como conclusión que las variables `perimeter y area` dependen fuertemente de `radius`.

Igualmente, podemos apreciar cómo las variables `concavity y compactness` dependen también en gran medida de `concave points`.

A continuación, realizaremos diversas observaciones de estos datos para ver si realmente las dependencias ya mencionadas se ven reflejadas en el conjunto de datos.

Ahora realizaremos 3 distintos **Mapas de Correlación**. La respuesta a esto es que separaremos los 3 tipos de variables `mean, se y worst` en mapas distintos, para poder visualizar de una manera más clara las correlaciones entre las variables de nuestro conjunto.

Comenzamos con el Mapa de Correlación de las variables de tipo `mean`:

In [ ]:
X_train_mean = X_train.loc[:,'radius_mean':'fractal_dimension_mean']

fig = px.imshow(X_train_mean.corr(),title="Mapa de Correlación variables 'mean'")
fig.show()

X_train_mean.corr()

Seguimos con el Mapa de Correlación de las variables de tipo `se`:

In [ ]:
X_train_se = X_train.loc[:,'radius_se':'fractal_dimension_se']

fig = px.imshow(X_train_se.corr(),title="Mapa de Correlación variables 'se'")
fig.show()

X_train_se.corr()

In [ ]:
X_train_worst = X_train.loc[:,'radius_worst':'fractal_dimension_worst']

fig = px.imshow(X_train_worst.corr(),title="Mapa de Correlación variables 'worst'")
fig.show()

X_train_worst.corr()

Estudiando los tres mapas de correlación podemos sacar una clara conclusión: **Las variables `perimeter y area` dependen directamente de `radius`.** Esto significa que más adelante en el preprocesamiento de datos podremos eliminar las variables `perimeter y area` sin problema.

De la misma manera, vemos cómo las variables `concavity y compactness` dependen también en gran medida de `concave points`, por lo que ambas variables podrán tambier ser eliminadas posteriormente, debido a su dependencia con esta última.

Mediante un **Boxplot** veremos de mejor forma como están distribuidos los datos de cada una, y así ademas identificar de mejor forma aquellas que tengan valores atípicos (*outliers*), no solo fijándonos en el histograma. Ahora, podremos fijarnos en cada variable de forma individual para comprobar si cuentan con algún outlier.

In [ ]:
f = go.Figure(data=[{'type': 'box', 'y': train[v], 'name': v} for v in set(train.columns)- {"diagnosis"}])
f.show()

Las conclusiones que podemos obtener de este diagrama es que encontramos **outliers** en las variables `radius, area, y perimeter` en sus tres variantes (como ya habíamos comentado en el histograma), y además las variables `smoothness y concavity` cuentan con valores atípicos.

Esto será algo que será algo a tener en cuenta a la hora de preprocesar los datos si queremos eliminarlos.

## 2. Preprocesamiento de datos

En esta etapa limpiaremos y organizaremos los datos de manera adecuada para entrenar a nuestro modelo basándonos en las observación que hemos realizado en el análisis exploratorio de datos previo. Por ello, en este conjunto de datos nos centraremos en la selección de variables adecuadas para conseguir reducir el número de estas.


Para realizar este proceso, haremos uso de un **Pipeline**. Este Pipeline será el encargado de aplicar las transformaciones que hemos decidido a nuestro conjunto de datos.

Como hemos decidido anteriormente graciás al análisis exploratorio de los datos, debemos eliminar las columnas (variables) seleccionadas previamente:
* Aquellas relacionadas con los `concave points`: `concavity_mean, compactness_mean, concavity_se, compactness_se, concavity_worst y compactness_worst`.
* Aquellas relacionadas con `radius`, es decir: `area_mean, perimeter_mean, area_se, perimeter_se, area_worst y perimeter_worst`.

La respuesta a por qué estas columnas son las mencionadas previamente, porque son variables que dependen directamente de las 2 que vamos a dejar en nuestro conjunto de datos: `radius y concave points`.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

preproc = ColumnTransformer([("", "drop", ["concavity_mean", "compactness_mean", "concavity_se", "compactness_se", "concavity_worst", "compactness_worst", "area_mean", "perimeter_mean", "area_se", "perimeter_se", "area_worst", "perimeter_worst"])], remainder="passthrough")

### Discretización

Con el objeto de obtener unos datos más simples y convertir las variables numéricas en intervalos vamos a discretizar esas variables, siendo en este caso, todas las variables predictoras.

Para simplificar los datos realizaremos un proceso de discretización de los datos. Esto quiere decir que convertiremos las variables que son numéricas (todas las variables predictoras que tengamos **después de la selección de variables** que hemos hecho previamente) en variables agrupadas por intervalos, lo que restará complejidad al modelo.

Al no poder observar puntos de corte claros, tomaremos arbitrariamente la decisión de realizar la discretización en tres intervalos de igual anchura.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=3, strategy="uniform")

## 3. Aprendizaje y Evaluación *Zero-R*

## Aprendizaje *Zero-R*

Lo que este algoritmo hará será aprender un clasificador que asigne a los casos del conjunto de test la clase predominante en el conjunto de entrenamiento (ya vimos que el problema era desbalanceado). Como veremos, en nuestro conjunto de datos concreto no destaca por su efectividad.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

## Evaluación *Zero-R*

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión y tasa de acierto.

In [ ]:
pipeline = make_pipeline(preproc, zero_r_model)

Aplicamos nuestro preprocesamiento al conjunto de entrenamiento y al conjunto de test:

In [ ]:
cfs = [] # Matriz de confusión para curva ROC

In [ ]:
utils.evaluate1(pipeline,
               X_train, X_test,
               y_train, y_test)

cfs.append(confusion_matrix(y_test, pipeline.predict(X_test)))

Como era de esperar, el modelo *Zero-R* obtiene malos resultados, pues solo predice la clase mayoritaria en el conjunto de entrenamiento, en este caso 0 (B).

## 4. Aprendizaje y Evaluación *Arbol de Decisión*

## Aprendizaje *Arbol de Decisión*

Una vez estudiado el algoritmo `Zero-R` probaremos un nuevo método, la creación de un árbol de decisión.

Para obtener este arbol de decisión, usaremos el estimador `DecisionTreeClassifier` de `scikit-learn`, sin olvidar fijar la semilla que definimos al principio de la libreta para asegurar que los experimentos sean reproducibles:

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Aplicamos el pipeline al árbol de decisión creado. Diferenciaremos en dos:
* Solo preprocesamiento (eliminación de variables que no son necesarias)
* Preprocesamiento y discretización

In [ ]:
preprocessed_tree_model = make_pipeline(preproc, tree_model)
preprocessed_discretized_tree_model = make_pipeline(preproc, discretizer, tree_model)

## Evaluación *Arbol de Decisión*

Vamos a ver los resultados del árbol de decisión sin el conjunto de datos discretizado:

In [ ]:
utils.evaluate1(preprocessed_tree_model,
               X_train, X_test,
               y_train, y_test)

cfs.append(confusion_matrix(y_test, preprocessed_tree_model.predict(X_test)))

Y con el conjunto de datos discretizado:

In [ ]:
utils.evaluate1(preprocessed_discretized_tree_model,
               X_train, X_test,
               y_train, y_test)

cfs.append(confusion_matrix(y_test, preprocessed_discretized_tree_model.predict(X_test)))

In [ ]:
go.Figure([go.Scatter(x=[0, 1], y=[0, 1], line={'dash': 'dash'}, name='Clasificador aleatorio'), go.Scatter(x=[cfs[i][0, 1] / (cfs[i][0, 0] + cfs[i][0, 1]) for i in range(len(cfs))], y=[cfs[i][1, 1] / (cfs[i][1, 1] + cfs[i][1, 0]) for i in range(len(cfs))], mode='markers', hovertext=['ZeroR', 'Árbol sin discretizar', 'Árbol discretizando'], name='Clasificadores propuestos')], layout={'title': 'Espacio ROC', 'xaxis': {'title': '1-specificity'}, 'yaxis': {'title': 'sensitivity'}})

Observando los resultados de los tres clasificadores, podemos decir que los árboles de decisión son superiores al ZeroR en *accuracy*, además de alcanzar un mejor compromiso entre *sensitivity* y *specificity*. Podemos observar estos valores individualmente también y vemos que si nos interesa optimizar el *recall*, que suele ser el caso con ciertos problemas médicos como este, el que mejor valor obtiene en esta métrica son los árbol entrenado sin discretizar previamente (puesto que el árbol discretizado nos aporta el mismo resultado). Por tanto en este problema podemos observar que la discretización previa no ha sido significativamente útil.

## Conclusión

Como puede resultar evidente, los árboles de decisión que hemos implementado obtienen mejores resultados que el algoritmo `Zero-R` puesto que no solo se quedan con la clase mayoritaria.

A su vez, es importante comentar que el árbol de decisión entrenado con el conjunto de datos discretizado (tras realizar las modificaciones necesarias que aprendimos en el análisis exploratorio) obtiene una tasa de acierto similar al árbol de decisión con el conjunto de datos sin discretizar para este problema concreto.

La conclusión que podemos obtener es que con un sencillo preprocesamiento y empleando un modelo predictivo no muy complejo obtendremos unas predicciones con una precisión de un 92%, lo que nos demuestra la importancia de preprocesar nuestro conjunto de datos.

---

# Titanic

Comenzamos cargando el conjunto de datos `titanic`:

Como es un conjunto de datos para una competición, nos quedaremos solo con la parte de train para el análisis exploratorio de los datos, puesto que es la única que contiene la variable predictora `Survived` que ahora comentaremos.

In [ ]:
filepath = "../input/titanic/train.csv"

index = "PassengerId"
target = "Survived"

data = utils.load_data(filepath, index, target)

Este conjunto de datos está compuesto por 10 variables predictoras que recogen datos acerca de los pasajeros del Titanic y la variable clase `Survived` que nos indicará si el pasajero murió (0) o sobrevivió (1).
Las variables predictoras, por tanto, son:
* `Pclass`: Estatus socio-económico de cada pasajero
* `Name`: Los nombres de los pasajeros
* `Sex`: Sexo de los pasajeros
* `Age`: Edad de los pasajeros
* `SibSp`: Siblings & Spouses. Número de hermanos y parejas que viajaban en el barco
* `Parch`: Número de parientes que cada pasajero tenía en el barco
* `Ticket`: Número de su ticket de embarque
* `Fare`: Precio de la tarifa que cada pasajero pagó
* `Cabin`: Camarote en los que se alojaban los pasajeros
* `Embarked`: Ciudad de embarque (Southampton, Cherburgo y Queenstown)

El conocimiento de estas variables nos ayudará a entender mejor como tratar los datos a la hora del análisis.

Una vez hemos cargado el conjunto de datos, mostraremos 5 registros aleatorios mediante la función `sample` para comprobar que el proceso ha sido realizado correctamente

In [ ]:
data.sample(5, random_state=seed)

In [ ]:
data.Survived.unique()

A continuación, separaremos en dos subconjuntos nuestro conjunto de datos inicial, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). 

In [ ]:
(X, y) = utils.divide_dataset(data, target="Survived")

Comprobaremos que se hayan separado correctamente, empezamos mostrando las variables predictoras:

In [ ]:
X.sample(5, random_state=seed)

Y a continuación la variable objetivo:

In [ ]:
y.sample(5, random_state=seed)

Antes de comenzar el análisis exploratorio de los datos, dividiremos nuestro conjunto de datos en otros dos subconjuntos, uno de entrenamiento y otro de prueba, con los siguientes porcentajes:

* Conjunto de entrenamiento: **70%**
* Conjunto de prueba: **30%**

Mediante este proceso, nos aseguraremos de que los resultados posteriores del proceso de validación han sido obtenidos de una manera correcta.

In [ ]:
train_size = 0.7
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Seguidamente, lo que haremos será unir los conjuntos `X_train` e `y_train` para obtener el conjunto de datos de entrenamiento.
Haremos los mismo para `X_test` e `y_test`, juntando así el conjunto de datos de test.

In [ ]:
train = utils.join_dataset(X_train, y_train)
test = utils.join_dataset(X_test, y_test)

Nuevamente, vamos a asegurarnos de que el conjunto de datos se ha dividido correctamente. Para ello, visualizaremos las variables del conjunto de datos de entrenamiento, observando que la variable clase también aparece al final del conjunto:

In [ ]:
train.sample(5, random_state=seed)

Una vez tenemos bien definidos nuestros conjuntos de entrenamiento y prueba, podemos pasar con el análisis exploratorio de datos.

## 1. Análisis exploratorio de datos

El análisis exploratorio de datos es un paso fundamental a la hora de comprender los datos con los que vamos a trabajar.

El objetivo de este análisis es explorar, describir y visualizar la naturaleza de los datos recogidos mediante la aplicación de técnicas simples de resumen de datos y métodos gráficos, para observar las posibles relaciones entre las variables de nuestro conjunto de datos.

Para comenzar, veremos una descripción del conjunto de datos que vamos a emplear.

### Descripción del conjunto de datos
El número de casos y variables (respectivamente) del conjunto de datos se puede obtener consultando el atributo `shape`:
Observamos como de los 891 registros iniciales, tenemos **623** de ellos para el entrenamiento (un 70%), y **268** (un 30%) para el conjunto de prueba, ambos con 11 variables, las 10 predictoras y la variable objetivo.

In [ ]:
data.shape

In [ ]:
train.shape

In [ ]:
test.shape

Para conocer cuál es el tipo de las variables, recurrimos al método `info`:

In [ ]:
train.info(memory_usage=False)

De acuerdo con esto, encotramos variedad entre las variables predictoras del conjunto de datos. Tenemos variables de tipo `object`, `int64`, y `float`. Sin embargo, la variable clase (`Survived`) es categórica y contiene dos estados, cuyos valores serán `0`y`1` como ya hemos comentado anteriormente:

Como vemos, encontramos valores nulos en ciertas variables. Veámoslo mejor de la siguiente manera:

In [ ]:
train.isnull().sum()

La variable `Cabin` contiene una gran cantidad de valores nulos, por lo que a la hora de visualizar las variables no debemos tenerla muy en cuenta, y en el posterior procesamiento será eliminda puesto que a primera vista no aporta información relevante.

In [ ]:
y_train.cat.categories

### Visualización de las variables

A la hora de visualizar las variables, comenzaremos comprobando la distribución de la variable clase de nuestro conjunto de entrenamiento. Como vemos, hay 384 instancias de la clase mayoritaria `0`, es decir que no sobrevivieron, y 239 de la clase `1`, es decir que sobrevivieron.

In [ ]:
train.describe(include="category")

Se puede observar mejor de la siguiente manera, comprobando gráficamente qué clase es la que predomina:

In [ ]:
utils.plot_barplot(train)

Lo que podemos observar es que las clases de la variable objetivo del problema no tienen el mismo número de casos, es decir, el problema está desbalanceado (las frecuencias de las combinaciones de estados no aparecen en la misma proporción).

Ahora que sabemos que el problema es **desbalanceado**, es el turno de visualizar las variables predictoras del conjunto de datos.

Comenzaremos mostrando un **histograma** con todas las variables, para mostrar la densidad de ejemplos para los distintos valores de las variables numéricas y analizar las tendencias que estas toman:

In [ ]:
utils.plot_histogram(train)

Mediante el estudio de este histograma en el que se han representado todas las variables, podremos extraer diversas conclusiones que nos serán útiles a la hora de preprocesar los datos de nuestro conjunto:
* Mirando las variables independientemente, encontramos outliers en las variables `SibSp`, ya que ciertas personas contaban con 8 familiares en el barco, lo que se aleja mucho de lo común. También la variable `Fare` tiene outliers, puesto que varias personas se alejan mucho de la tarifa media (unos 15), llegando hasta valores de más de 500. Esto será algo a tener en cuenta a la hora de preprocesar los datos si queremos eliminarlos.
* Estimación de las distribuciones:
    * Si analizamos la variable `age`, veremos que sigue una distribución normal con tendencia a la derecha (mayor edad).
    * Si analizamos las variables `SibSp y Parch`, veremos que siguen una distribución exponencial.

Ahora realizaremos un análisis multivariado para intentar encontrar correlaciones entre las variables.

Para ello, obtenemos del conjunto de entrenamiento todas las variables pese a que algunas son más representativas que otras. Con ellas, realizaremos una matriz de dispersión por parejas para comprobar las relaciones entre ellas.

Como podemos ver, la variable clase se diferencia también en nuestra gráfica, siendo el color naranja los no supervivientes `0`, y su homólogo azul los supervivientes `1`:

In [ ]:
fig = utils.plot_pairplot(train, 'Survived')
fig.update_layout(width=1200, height=1000)

Estudiando la matriz de dispersión de las variables predictoras ordenadas por parejas, podemos sacar como conclusión que a primera vista ninguna de las variables está directamente relacionada con otra, por lo que cada una aporta una información distinta.

A continuación, realizaremos diversas observaciones de estos datos para ver si se nos hubiesen escapado dependencias en el conjunto de datos.

Ahora realizaremos un **mapa de correlación** para poder visualizar de una manera más clara las correlaciones entre las variables numéricas de nuestro conjunto.

In [ ]:
px.imshow(train.corr())

In [ ]:
train.corr()

Estudiando el mapa de correlación podemos sacar una clara conclusión: Ninguna variable depende directamente de otra, como ya habíamos observado en las matrices de dispersión por parejas.

En el posterior preprocesamiento, esto implicará que si deseamos eliminar alguna variable no será debido a su dependencia de otra, si no a otras características que comentaremos más adelante.

Mediante un **Boxplot** veremos de mejor forma como están distribuidos los datos de cada variable predictora, y así ademas identificar de mejor forma aquellas que tengan valores atípicos (*outliers*), no solo fijándonos en el histograma. Ahora, podremos fijarnos en cada variable de forma individual para comprobar si cuentan con algún outlier.

In [ ]:
f = go.Figure(data=[{'type': 'box', 'y': train[v], 'name': v} for v in set(train.columns)- {"Survived"}])
f.show()

Si nos centramos en cada variable de manera aislada, podemos observar lo siguiente:

En cuanto a las variables como `Cabin, Sex, Name, Pclass y Ticket`, no podemos considerar outliers puesto que son variables con valores totalmente distintos y por ello no podrán contar con valores atípicos.

Otra de las conclusiones que podemos obtener de este diagrama es que encontramos **outliers** en las variables `SibSp, Fare, y Embarked` como ya habíamos comentado en el histograma. Esto es interesante, puesto que nos indica que la mayoría de gente embarcó en Southampton, y se considera a los otros lugares de embarque como valores atípicos.

Esto será algo que será algo a tener en cuenta a la hora de preprocesar los datos si queremos eliminarlos.

## 2. Preprocesamiento de datos

En esta etapa limpiaremos y organizaremos los datos de manera adecuada para entrenar a nuestro modelo basándonos en las observación que hemos realizado en el análisis exploratorio de datos previo. Por ello, en este conjunto de datos nos centraremos en la selección de variables adecuadas para conseguir reducir el número de estas.


Para realizar este proceso, haremos uso de un **Pipeline**. Este Pipeline será el encargado de aplicar las transformaciones que hemos decidido a nuestro conjunto de datos.

Gracias al análisis exploratorio de los datos, podemos eliminar diversas columnas que no aportan apenas información relevante:
* `Name, Cabin, Ticket`.
La respuesta a por qué estas columnas, es porque son variables que aportan información diferente para cada instancia del conjunto de datos, por lo que no aporta información relevamente. Además como hemos comentado previamente, `Cabin` cuenta con multitud de valores nulos.

Además, aplicaremos una imputación para las variables con valores perdidos que se vieron en el análisis exploratorio y que permanecerán en el conjunto de datos como `Age y Embarked`. El método de imputación a seguir será por moda (most_frequent), de modo que los valores perdidos se verán sustituidos por la moda.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

moda_imp = SimpleImputer(strategy='most_frequent')
preproc = ColumnTransformer([("", "drop", ["Name", "Cabin", "Ticket"]),('moda_inp', moda_imp, ["Age", "Embarked"])], remainder="passthrough")

### Eliminación de outliers

Para la detección y eliminación de los outliers emplearemos un ensemble llamado `IsolationForest` de scikit-learn. Cuando éste detecte un outlier, borrará toda la instancia.

In [ ]:
def eliminar_outliers(X, y=None, random_state=0):
    model = IsolationForest(random_state=random_state)
    model.fit(X)
    y_pred = model.predict(X)
    if y!=None:
        return X[y_pred == 1], y[y_pred == 1]
    else:
        return X[y_pred == 1]

eliminar_outliers = FunctionTransformer(func=eliminar_outliers, kw_args={"random_state":seed}, accept_sparse=True)


### Codificación de valores

Mediante un codificación **one-hot**, podremos trabajar cómodamente con las variables categóricas y objetos a la hora de crear los algoritmos con los que evaluaremos el conjunto de datos.

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')

Una vez hemos realizado nuestro preprocesamiento de datos, es hora de pasar a aprender los modelos y posteriormente evaluarlos haciendo uso del procesamiento que hemos llevado a cabo, lo que implica añadir al **pipeline** el preprocesamiento (eliminación de variables e imputación de valores), la codificación one-hot y la eliminación de outliers.

## 3. Aprendizaje y Evaluación *Zero-R*

## Aprendizaje *Zero-R*

Lo que este algoritmo hará será aprender un clasificador que asigne a los casos del conjunto de test la clase predominante en el conjunto de entrenamiento (ya vimos que el problema era desbalanceado). Como veremos, en nuestro conjunto de datos concreto no destaca por su efectividad.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

## Evaluación *Zero-R*

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión y tasa de acierto.

In [ ]:
pipeline = make_pipeline(preproc, encoder, eliminar_outliers, zero_r_model)

Aplicamos nuestro preprocesamiento al conjunto de entrenamiento y al conjunto de test:

In [ ]:
cfs = [] # Matriz de confusión para curva ROC

In [ ]:
utils.evaluate(pipeline,
               X_train, X_test,
               y_train, y_test)

cfs.append(confusion_matrix(y_test, pipeline.predict(X_test)))

Como era de esperar, el modelo *Zero-R* obtiene malos resultados, pues solo predice la clase mayoritaria en el conjunto de entrenamiento, en este caso 0 (No supervivientes).

## 4. Aprendizaje y Evaluación *Arbol de Decisión*

## Aprendizaje *Arbol de Decisión*

Una vez estudiado el algoritmo `Zero-R` probaremos un nuevo método, la creación de un árbol de decisión.

Para obtener este arbol de decisión, usaremos el estimador `DecisionTreeClassifier` de `scikit-learn`, sin olvidar fijar la semilla que definimos al principio de la libreta para asegurar que los experimentos sean reproducibles:

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Aplicamos el pipeline al árbol de decisión creado con el preprocesamiento aplicado.

In [ ]:
preprocessed_tree_model = make_pipeline(preproc, encoder, eliminar_outliers, tree_model)

## Evaluación *Arbol de Decisión*

Vamos a ver los resultados del árbol de decisión:

In [ ]:
utils.evaluate(preprocessed_tree_model,
               X_train, X_test,
               y_train, y_test)

cfs.append(confusion_matrix(y_test, preprocessed_tree_model.predict(X_test)))

In [ ]:
go.Figure([go.Scatter(x=[0, 1], y=[0, 1], line={'dash': 'dash'}, name='Clasificador aleatorio'), go.Scatter(x=[cfs[i][0, 1] / (cfs[i][0, 0] + cfs[i][0, 1]) for i in range(len(cfs))], y=[cfs[i][1, 1] / (cfs[i][1, 1] + cfs[i][1, 0]) for i in range(len(cfs))], mode='markers', hovertext=['ZeroR', 'Árbol sin discretizar', 'Árbol discretizando'], name='Clasificadores propuestos')], layout={'title': 'Espacio ROC', 'xaxis': {'title': '1-specificity'}, 'yaxis': {'title': 'sensitivity'}})

Observando los resultados de los dos clasificadores, podemos decir que los árboles de decisión son superiores al ZeroR en *accuracy*, además de alcanzar un mejor compromiso entre *sensitivity* y *specificity*. Como en los conjuntos de datos estudiados previamente en esta libreta, hemos optado por eliminar la discretización previa puesto que no hemos encontrado que haya sido significativamente útil.

## Conclusión

Como puede resultar evidente, el árbol de decisión que hemos implementado obtiene mejores resultados que el algoritmo `Zero-R` puesto que no solo se quedan con la clase mayoritaria.

A su vez, es importante comentar que el árbol de decisión entrenado con el conjunto de datos sin discretizar nos proporciona un recall no excesivamente bueno para el problema que estamos tratando. Lo que si podemos ver es que empleando un modelo predictivo no muy complejo obtendremos unas predicciones mejores que sin él, lo que nos demuestra la importancia de preprocesar nuestro conjunto de datos.